In [1]:
from omegaconf import OmegaConf
config = OmegaConf.load("D:\\Nik\\Projects\\mlfmf-poskusi\\code2seq-config.yaml")

In [2]:
from code2seq.model import Code2Seq
model = Code2Seq.load_from_checkpoint("D:\\Nik\\Projects\\mlfmf-poskusi\\lightning_logs\\version_37\\checkpoints\\epoch=9-step=199.ckpt")

In [3]:
from code2seq.data.path_context_data_module import PathContextDataModule
data_module = PathContextDataModule(
    config.data_folder,
    config.data
)

In [4]:
model

Code2Seq(
  (_Code2Seq__metrics): MetricCollection(
    (test_chrf): ChrF()
    (test_f1): SequentialF1Score()
    (train_f1): SequentialF1Score()
    (val_chrf): ChrF()
    (val_f1): SequentialF1Score()
  )
  (_encoder): PathEncoder(
    (token_embedding): Embedding(1195, 128, padding_idx=0)
    (node_embedding): Embedding(42, 128, padding_idx=0)
    (dropout_rnn): Dropout(p=0.25, inplace=False)
    (path_lstm): LSTM(128, 128, bidirectional=True)
    (embedding_dropout): Dropout(p=0.25, inplace=False)
    (linear): Linear(in_features=512, out_features=128, bias=False)
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (_decoder): Decoder(
    (_decoder_step): LSTMDecoderStep(
      (_target_embedding): Embedding(429, 128, padding_idx=0)
      (_attention): LuongAttention(
        (attn): Linear(in_features=128, out_features=128, bias=False)
      )
      (_decoder_lstm): LSTM(128, 128, batch_first=True)
      (_dropout_rnn): Dropout(p=0.5, inplace=False)
      (_c

In [5]:
predict_data_loader = data_module.predict_dataloader()

  0%|          | 0/3280 [00:00<?, ?it/s]

In [6]:
sample = next(iter(predict_data_loader))

In [7]:
sample.from_token.shape

torch.Size([7, 5626])

In [8]:
output_logits, attention_weights = model.forward(sample.from_token, sample.path_nodes, sample.to_token, sample.contexts_per_label, sample.labels.shape[0])

In [9]:
output_logits.shape

torch.Size([8, 766, 429])

In [10]:
attention_weights.shape

torch.Size([8, 766, 142])

In [11]:
output_logits[0].shape

torch.Size([766, 429])

In [12]:
prediction = output_logits.argmax(-1)

In [13]:
prediction

tensor([[2, 2, 2,  ..., 2, 2, 2],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [14]:
predictions = []
for sample in predict_data_loader:
    output_logits, _ = model.forward(sample.from_token, sample.path_nodes, sample.to_token, sample.contexts_per_label, sample.labels.shape[0])
    predictions.append(output_logits.argmax(-1))

In [15]:
len(predictions)

5